In [23]:
import os
import pandas as pd
import numpy as np
import boto3
import awswrangler as wr

In [24]:
# Abres un cliente de S3
session = boto3.Session(profile_name='arquitectura')
s3 = session.client('s3')

In [25]:
# Set up the AWS Glue client
from botocore.config import Config
my_config = Config(
    region_name = 'us-east-1'
)

glue = boto3.client('glue', config=my_config)

In [3]:
# Cracion de la base de datos
response = glue.create_database(
    DatabaseInput={
        'Name': 'econ',
        'Description': 'DB para la tarea 6.',
    },
)
response

NoCredentialsError: Unable to locate credentials

Lo tuve que hacer desde la consola de AWS, no encontré las credenciales.

In [17]:
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`inflacion` (`fecha` date, `dato` float) 
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' 
    WITH SERDEPROPERTIES ('field.delim' = ',') 
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat' 
    LOCATION 's3://itam-analytics-carlos/raw/inflacion/' 
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

In [18]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

""


In [26]:
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`tasa_de_interes` (`fecha` date, `dato` float) 
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' 
    WITH SERDEPROPERTIES ('field.delim' = ',') 
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat' 
    LOCATION 's3://itam-analytics-carlos/raw/tasa_de_interes/' 
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

In [27]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

""


In [28]:
query = '''
    CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`tipo_de_cambio` (`fecha` date, `dato` float) 
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' 
    WITH SERDEPROPERTIES ('field.delim' = ',') 
    STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat' 
    LOCATION 's3://itam-analytics-carlos/raw/tipo_de_cambio/' 
    TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

In [29]:
wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False, 
    boto3_session=session
)

""


In [35]:
query = '''
    CREATE TABLE IF NOT EXISTS econ.ind AS (
        SELECT 
            a.fecha,
            a.dato AS tdi,
            b.dato AS tdc,
            c.dato AS inf
        FROM econ.tasa_de_interes AS a
        INNER JOIN econ.tipo_de_cambio AS b ON a.fecha = b.fecha
        INNER JOIN econ.inflacion AS c ON MONTH(a.fecha) = MONTH(c.fecha)
    )
'''

wr.athena.read_sql_query(
    query, 
    database="econ", 
    ctas_approach=False,
    boto3_session=session
)

""
